<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Yvonne", "transactions": [{"transaction-id": 214, "amount": 1180}, {"transaction-id": 486, "amount": 1129}, {"transaction-id": 970, "amount": 1178}, {"transaction-id": 1076, "amount": 1154}, {"transaction-id": 1664, "amount": 1143}, {"transaction-id": 1915, "amount": 1140}, {"transaction-id": 1971, "amount": 1128}, {"transaction-id": 2122, "amount": 1159}, {"transaction-id": 2585, "amount": 1257}, {"transaction-id": 2654, "amount": 1243}, {"transaction-id": 2765, "amount": 1192}, {"transaction-id": 3112, "amount": 1215}, {"transaction-id": 3302, "amount": 1088}, {"transaction-id": 3348, "amount": 1223}, {"transaction-id": 3691, "amount": 1246}, {"transaction-id": 3897, "amount": 1143}, {"transaction-id": 4044, "amount": 1243}, {"transaction-id": 4143, "amount": 1168}, {"transaction-id": 4220, "amount": 1200}, {"transaction-id": 4357, "amount": 1211}, {"transaction-id": 4485, "amount": 1192}, {"transaction-id": 4618, "amount": 1189}, {"transaction-id": 4661, "amount

In [5]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Yvonne", "transactions": [{"transaction-id": 214, "amount": 1180}, {"transaction-id": 486, "amount": 1129}, {"transaction-id": 970, "amount": 1178}, {"transaction-id": 1076, "amount": 1154}, {"transaction-id": 1664, "amount": 1143}, {"transaction-id": 1915, "amount": 1140}, {"transaction-id": 1971, "amount": 1128}, {"transaction-id": 2122, "amount": 1159}, {"transaction-id": 2585, "amount": 1257}, {"transaction-id": 2654, "amount": 1243}, {"transaction-id": 2765, "amount": 1192}, {"transaction-id": 3112, "amount": 1215}, {"transaction-id": 3302, "amount": 1088}, {"transaction-id": 3348, "amount": 1223}, {"transaction-id": 3691, "amount": 1246}, {"transaction-id": 3897, "amount": 1143}, {"transaction-id": 4044, "amount": 1243}, {"transaction-id": 4143, "amount": 1168}, {"transaction-id": 4220, "amount": 1200}, {"transaction-id": 4357, "amount": 1211}, {"transaction-id": 4485, "amount": 1192}, {"transaction-id": 4618, "amount": 1189}, {"transaction-id": 4661, "amount

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Yvonne',
  'transactions': [{'transaction-id': 214, 'amount': 1180},
   {'transaction-id': 486, 'amount': 1129},
   {'transaction-id': 970, 'amount': 1178},
   {'transaction-id': 1076, 'amount': 1154},
   {'transaction-id': 1664, 'amount': 1143},
   {'transaction-id': 1915, 'amount': 1140},
   {'transaction-id': 1971, 'amount': 1128},
   {'transaction-id': 2122, 'amount': 1159},
   {'transaction-id': 2585, 'amount': 1257},
   {'transaction-id': 2654, 'amount': 1243},
   {'transaction-id': 2765, 'amount': 1192},
   {'transaction-id': 3112, 'amount': 1215},
   {'transaction-id': 3302, 'amount': 1088},
   {'transaction-id': 3348, 'amount': 1223},
   {'transaction-id': 3691, 'amount': 1246},
   {'transaction-id': 3897, 'amount': 1143},
   {'transaction-id': 4044, 'amount': 1243},
   {'transaction-id': 4143, 'amount': 1168},
   {'transaction-id': 4220, 'amount': 1200},
   {'transaction-id': 4357, 'amount': 1211},
   {'transaction-id': 4485, 'amount': 1192},
   {'transa

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 28,
  'name': 'Alice',
  'transactions': [{'transaction-id': 11, 'amount': 1047},
   {'transaction-id': 346, 'amount': 1650},
   {'transaction-id': 429, 'amount': 1667},
   {'transaction-id': 765, 'amount': 1793},
   {'transaction-id': 875, 'amount': 1452},
   {'transaction-id': 1034, 'amount': 1537},
   {'transaction-id': 1148, 'amount': 1177},
   {'transaction-id': 1245, 'amount': 1545},
   {'transaction-id': 1273, 'amount': 1485},
   {'transaction-id': 1397, 'amount': 1371},
   {'transaction-id': 1745, 'amount': 1065},
   {'transaction-id': 1905, 'amount': 1268},
   {'transaction-id': 1968, 'amount': 1337},
   {'transaction-id': 2043, 'amount': 1036},
   {'transaction-id': 2363, 'amount': 1441},
   {'transaction-id': 2391, 'amount': 1250},
   {'transaction-id': 2664, 'amount': 1216},
   {'transaction-id': 2951, 'amount': 1110},
   {'transaction-id': 3130, 'amount': 1448},
   {'transaction-id': 3384, 'amount': 1676},
   {'transaction-id': 3492, 'amount': 1538},
   {'transacti

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 53},
 {'name': 'Alice', 'count': 28},
 {'name': 'Alice', 'count': 8},
 {'name': 'Alice', 'count': 36},
 {'name': 'Alice', 'count': 4})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(53, 28, 8, 36, 4)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

21.496610169491525

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

([{'transaction-id': 11, 'amount': 1047},
  {'transaction-id': 346, 'amount': 1650},
  {'transaction-id': 429, 'amount': 1667},
  {'transaction-id': 765, 'amount': 1793},
  {'transaction-id': 875, 'amount': 1452},
  {'transaction-id': 1034, 'amount': 1537},
  {'transaction-id': 1148, 'amount': 1177},
  {'transaction-id': 1245, 'amount': 1545},
  {'transaction-id': 1273, 'amount': 1485},
  {'transaction-id': 1397, 'amount': 1371},
  {'transaction-id': 1745, 'amount': 1065},
  {'transaction-id': 1905, 'amount': 1268},
  {'transaction-id': 1968, 'amount': 1337},
  {'transaction-id': 2043, 'amount': 1036},
  {'transaction-id': 2363, 'amount': 1441},
  {'transaction-id': 2391, 'amount': 1250},
  {'transaction-id': 2664, 'amount': 1216},
  {'transaction-id': 2951, 'amount': 1110},
  {'transaction-id': 3130, 'amount': 1448},
  {'transaction-id': 3384, 'amount': 1676},
  {'transaction-id': 3492, 'amount': 1538},
  {'transaction-id': 3743, 'amount': 1233},
  {'transaction-id': 3840, 'amount': 1

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 11, 'amount': 1047},
 {'transaction-id': 346, 'amount': 1650},
 {'transaction-id': 429, 'amount': 1667})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(1047, 1650, 1667)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

675.0178979736655

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
is_even = lambda x: x % 2
b.foldby(is_even, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 141), ('Alice', 142), ('Alice', 152), ('Alice', 155), ('Bob', 96), ('Bob', 96), ('Bob', 104), ('Bob', 104), ('Charlie', 131), ('Charlie', 132), ('Charlie', 142), ('Charlie', 143), ('Dan', 96), ('Dan', 96), ('Dan', 104), ('Dan', 104), ('Edith', 183), ('Edith', 183), ('Edith', 192), ('Edith', 194), ('Frank', 84), ('Frank', 84), ('Frank', 91), ('Frank', 91), ('George', 95), ('George', 96), ('George', 103), ('George', 104), ('Hannah', 106), ('Hannah', 108), ('Hannah', 117), ('Hannah', 117), ('Ingrid', 60), ('Ingrid', 60), ('Ingrid', 65), ('Ingrid', 65), ('Jerry', 156), ('Jerry', 156), ('Jerry', 169), ('Jerry', 169), ('Kevin', 180), ('Kevin', 180), ('Kevin', 194), ('Kevin', 194), ('Laura', 69), ('Laura', 71), ('Laura', 72), ('Laura', 74), ('Michael', 132), ('Michael', 132), ('Michael', 141), ('Michael', 143), ('Norbert', 108), ('Norbert', 108), ('Norbert', 115), ('Norbert', 116), ('Oliver', 48), ('Oliver', 48), ('Oliver', 52), ('Oliver', 52), ('Patricia', 108), ('Patricia', 117),

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot+1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 590), ('Bob', 400), ('Charlie', 548), ('Dan', 400), ('Edith', 752), ('Frank', 350), ('George', 398), ('Hannah', 448), ('Ingrid', 250), ('Jerry', 650), ('Kevin', 748), ('Laura', 286), ('Michael', 548), ('Norbert', 447), ('Oliver', 200), ('Patricia', 450), ('Quinn', 599), ('Ray', 231), ('Sarah', 300), ('Tim', 546), ('Ursula', 457), ('Victor', 406), ('Wendy', 747), ('Xavier', 450), ('Yvonne', 756), ('Zelda', 284)]
CPU times: user 134 ms, sys: 7.92 ms, total: 142 ms
Wall time: 573 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Yvonne,"[{'transaction-id': 214, 'amount': 1180}, {'tr..."
1,1,Victor,"[{'transaction-id': 76, 'amount': 645}, {'tran..."
2,2,Edith,"[{'transaction-id': 5, 'amount': 163}, {'trans..."
3,3,Yvonne,"[{'transaction-id': 266, 'amount': 3515}, {'tr..."
4,4,Tim,"[{'transaction-id': 74, 'amount': 125}, {'tran..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 187 ms, sys: 9.47 ms, total: 196 ms
Wall time: 1.66 s


name
Alice      590
Bob        400
Charlie    548
Dan        400
Edith      752
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Yvonne', 'amount': 1180, 'transaction-id': 214},
 {'id': 0, 'name': 'Yvonne', 'amount': 1129, 'transaction-id': 486},
 {'id': 0, 'name': 'Yvonne', 'amount': 1178, 'transaction-id': 970})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Yvonne,1180,214
1,0,Yvonne,1129,486
2,0,Yvonne,1178,970
3,0,Yvonne,1154,1076
4,0,Yvonne,1143,1664


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 169 ms, sys: 15.1 ms, total: 184 ms
Wall time: 1.42 s


name
Alice       12683
Bob         15429
Charlie     17945
Dan         16735
Edith       23076
Frank        8924
George      19957
Hannah      15804
Ingrid      15671
Jerry       26170
Kevin       37605
Laura       13635
Michael     11229
Norbert     18613
Oliver      14116
Patricia    18107
Quinn       19645
Ray          7088
Sarah       11913
Tim         22050
Ursula      24899
Victor      13689
Wendy       30369
Xavier      13771
Yvonne      48593
Zelda       22284
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()